In [1]:
# sys, file and nav packages:
import datetime as dt
import json

# math packages:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.distributions.empirical_distribution import ECDF

# charting:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import ticker
from matplotlib import colors
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.gridspec import GridSpec
import seaborn as sns

import base64, io, IPython
from PIL import Image as PILImage
from IPython.display import Markdown as md
from IPython.display import display, Math, Latex

# set some parameters:
today = dt.datetime.now().date().strftime("%Y-%m-%d")
start_date = '2020-03-01'
end_date ='2021-05-31'

a_fail_rate = 100

unit_label = 'p/100m'
reporting_unit = 100

# name of the output folder:
name_of_project = 'dist_map'

# get your data:
df= pd.read_csv('checked_sdata_eos_2020_21.csv')

with open("river_basins.json", "r") as infile:
    river_bassins = json.load(infile)

dfBeaches = pd.read_csv("beaches_with_land_use_rates.csv")
dfCodes = pd.read_csv("codes_with_group_names_2015.csv")
dfDims = pd.read_csv("corrected_dims.csv")

# set the index of the beach data to location slug
dfBeaches.set_index('slug', inplace=True)
        
dfCodes.set_index("code", inplace=True)

In [2]:
# this is the aggregated survey data that is being used
fd = df.copy()
fd['loc_dates'] = list(zip(fd.location.values, fd.date.values))
fd['date'] = pd.to_datetime(fd.date)

# the location data for the surveys
dfb = dfBeaches.loc[fd.location.unique()].copy()

In [3]:
# the landuse descriptors are integers, key them to a readable descriptor
# no27 is the results of intersecting buffer to Land use
label_keys = {
    1:"industrial",
    2:"residential",
    3:"government",
    4:"agg_buildings",
    5:"unk_building",
    6:"roads",
    7:"railways",
    8:"airports",
    9:"special",
    10:"recreational",
    11:"orchards",
    12:"vineyards",
    13:"horticulture",
    14:"arable",
    15:"meadows",
    16:"farmpastures",
    17:"alpinemeadows",
    18:"aplinepasteurs",
    19:"closed_forest",
    20:"open_forest",
    21:"brush_forest",
    22:"woods",
    23:"lakes",
    24:"rivers",
    25:"unproductive",
    26:"bareland",
    27:"glaciers"
}

# group the land use into functional groups
group_parts = {'buildings':[1,2,3,4,5,9],
               'trans':[6,7,8],
               'recreation':[10],
               'agg':[11, 12, 13, 14, 15, 16, 18],
               'woods':[17,19,20,21,22],
               'water':[23,24],
               'unproductive':[25,26,27]
              }

# make column names based on the key values:
as_1827_part ={k:F"part_{v}" for k,v in label_keys.items()}

# map survey results to these aggregated luse features
som_cols = ['% to buildings',
            '% to trans',
            '% to recreation',
            '% to agg',
            '% to woods',
            '% to water',
            '% to unproductive',
          
]

# define functions to map luse data to survey locations
def aggregate_buffer_data(data, cols, bufferdata, label_keys, **kwargs):
    
    # make empty columns for the measured land use features
    for acol in cols:
        data[acol]= 0

    # assign values to each column
    for beach in data.index:
        for label in list(label_keys.values()):        
            try:
                new_data = bufferdata[(bufferdata.slug == beach)&(bufferdata.label == label)].AS18_27.values[0]
            except:
                new_data = 0
            data.loc[beach, label] = new_data
    return data

# account for the area attributed to water by removing the
# the value of water features from land use total
def adjusted_land_use(data, col_keys):
    # total land use
    data['luse_total'] = data.loc[:,list(col_keys.values())].sum(axis=1)

    # amount attributed to water
    data['water_value'] = data.loc[:, ['lakes','rivers']].sum(axis=1)

    # the adjsuted land use
    # remove the water value from land use stats
    data['adjusted_land_use'] = data.luse_total - data.water_value
    
    return data

# determine the ration of each landuse feature to the adjusted total for each buffer
def account_for_adj_luse(data, col_keys):
    for label in list(col_keys.values()):
        a_label = F"part_{label}"
        data[a_label] = data[label]/data['adjusted_land_use']
    return data

# aggregate the groups
def aggregate_the_luse_groups(data, groups, group_parts, as_1827_part):
    for a_group in group_parts.keys():
        part_groups = [as_1827_part[x] for x in group_parts[a_group]]
        new_group = F"% to {a_group}"
        data[new_group] = data.loc[:,part_groups].sum(axis=1)
    
    return data

# merge survey results to land use stats
def assign_luse_stat_to_survey_results(sdata, luse_data, som_cols):
    for a_beach in sdata.location.unique():
        for element in som_cols:
            sdata.loc[sdata.location == a_beach, element] = luse_data.loc[a_beach, element]
    return sdata

# apply method and check results
def check_hypothesis(this_data, some_codes, variables, method):
    myresults = {}
    for i,code in enumerate(some_codes):
        data = this_data[this_data.code == code]
        code_results ={code:{}}
        for j, n in enumerate(variables):
            corr, a_p = method(data[n], data[unit_label])
            if a_p <= 0.05:
                code_results[code].update({n:corr})
        myresults.update(code_results)
    
    return myresults

### 2000 meters codes with an association to a landuse feature

Where p <= 0.05 for Spearmans test for association.

In [4]:
# the buffer results from qgis
lu_2000 = pd.read_csv('luse_2000.csv')
lu_2000.rename(columns={'location':'slug'}, inplace=True)

# make a df with location slug, landuse label and land use value:
for a_num in lu_2000.AS18_27.unique():
    lu_2000.loc[lu_2000.AS18_27==a_num, "label"]=label_keys[a_num]

# the locations that need land use data
data = pd.DataFrame(index = fd.location.unique())

# records with a quantity greater than 30
abundant_codes = fd[fd.quantity > 30].code.unique()

# getting a list of the current landuse categories
add_these_cols = lu_2000.label.unique()

# counting the number of points inside the buffer for each category and location
bufferdata = lu_2000.groupby(['slug','label'], as_index=False).AS18_27.count()

these_groups = list(group_parts.keys())

some_data = aggregate_buffer_data(data, add_these_cols, bufferdata, label_keys)

sd_0 = adjusted_land_use(some_data, label_keys)

sd_1 = account_for_adj_luse(data, label_keys)

sd_2 = aggregate_the_luse_groups(data, add_these_cols, group_parts, as_1827_part)

fd_luse = assign_luse_stat_to_survey_results(fd, sd_2,som_cols)    

this_data = fd_luse[[unit_label,*som_cols, 'code']]

myresults = check_hypothesis(this_data, abundant_codes, som_cols, stats.spearmanr)

srho_results_2000 = pd.DataFrame.from_dict(myresults, orient='index')
srho_results_2000.fillna("X", inplace=True)

# the survey data with land use attached
srho_results_2000 

,% to buildings,% to trans,% to recreation,% to agg,% to woods,% to water,% to unproductive
G27,0.333677,0.359557,0.302469,-0.284461,-0.164146,0.123172,-0.298779
G95,0.133874,0.187697,0.195286,-0.151915,X,0.124518,-0.169153
G30,0.259606,0.277116,0.275198,-0.202422,-0.133838,X,X
G67,-0.128185,X,X,X,X,X,X
G112,0.102842,0.105651,0.156375,X,X,X,X
G200,0.191516,X,0.11617,-0.21987,X,0.241411,X
G178,0.361203,0.35621,0.257235,-0.310423,-0.126493,X,-0.326207
G25,0.134717,0.110049,0.134953,-0.10919,X,0.130293,X
G98,0.103159,0.181116,0.215513,-0.172881,X,X,X
G73,0.129071,0.103307,X,X,X,X,X


### 2500 meters codes with an association to a landuse feature

Where p <= 0.05 for Spearmans test for association.

In [5]:
# the buffer results from qgis
new_buffer = pd.read_csv('luse_2500.csv')
new_buffer.rename(columns={'location':'slug'}, inplace=True)

# make a df with location slug, landuse label and land use value:
for a_num in new_buffer.AS18_27.unique():
    new_buffer.loc[new_buffer.AS18_27==a_num, "label"]=label_keys[a_num]

# the locations that need land use data
data = pd.DataFrame(index = fd.location.unique())

# records with a quantity greater than 30
abundant_codes = fd[fd.quantity > 30].code.unique()

# getting a list of the current landuse categories
add_these_cols = lu_2000.label.unique()

# counting the number of points inside the buffer for each category and location
bufferdata = new_buffer.groupby(['slug','label'], as_index=False).AS18_27.count()

some_data = aggregate_buffer_data(data, add_these_cols, bufferdata, label_keys)

sd_0 = adjusted_land_use(some_data, label_keys)

sd_1 = account_for_adj_luse(data, label_keys)

sd_2 = aggregate_the_luse_groups(data, add_these_cols, group_parts, as_1827_part)

fd_luse = assign_luse_stat_to_survey_results(fd, sd_2,som_cols)    

this_data = fd_luse[[unit_label,*som_cols, 'code']]

myresults = check_hypothesis(this_data, abundant_codes, som_cols, stats.spearmanr)
    

srho_results_2500 = pd.DataFrame.from_dict(myresults, orient='index')
srho_results_2500.fillna("X", inplace=True)

# the survey data with land use attached
srho_results_2500

,% to buildings,% to trans,% to recreation,% to agg,% to woods,% to unproductive,% to water
G27,0.338808,0.353151,0.289665,-0.300856,-0.160632,-0.331526,X
G95,0.151636,0.191291,0.169331,-0.138672,X,-0.20954,0.152909
G30,0.259458,0.273768,0.263673,-0.17438,-0.114198,-0.130139,0.102261
G67,-0.123343,X,X,X,0.104952,X,X
G112,0.133958,X,0.153405,X,-0.111106,X,X
G200,0.157847,0.111638,0.118095,-0.235673,X,X,0.221714
G178,0.357067,0.344676,0.260189,-0.307018,-0.132225,-0.320133,X
G25,0.12763,0.116756,0.12492,X,X,X,0.116392
G98,0.110209,0.176887,0.202598,-0.180012,X,-0.116251,X
G73,0.135705,0.122102,X,X,X,-0.109623,X


### 5000 meters codes with an association to a landuse feature

Where p <= 0.05 for Spearmans test for association.

In [6]:
# the buffer results from qgis
lu_2000 = pd.read_csv('luse_5k.csv')
lu_2000.rename(columns={'location':'slug'}, inplace=True)

# make a df with location slug, landuse label and land use value:
for a_num in lu_2000.AS18_27.unique():
    lu_2000.loc[lu_2000.AS18_27==a_num, "label"]=label_keys[a_num]

# the locations that need land use data
data = pd.DataFrame(index = fd.location.unique())

# records with a quantity greater than 30
abundant_codes = fd[fd.quantity > 30].code.unique()

# getting a list of the current landuse categories
add_these_cols = lu_2000.label.unique()

# counting the number of points inside the buffer for each category and location
bufferdata = lu_2000.groupby(['slug','label'], as_index=False).AS18_27.count()

these_groups = list(group_parts.keys())

some_data = aggregate_buffer_data(data, add_these_cols, bufferdata, label_keys)

sd_0 = adjusted_land_use(some_data, label_keys)

sd_1 = account_for_adj_luse(data, label_keys)

sd_2 = aggregate_the_luse_groups(data, add_these_cols, group_parts, as_1827_part)

fd_luse = assign_luse_stat_to_survey_results(fd, sd_2,som_cols)    

this_data = fd_luse[[unit_label,*som_cols, 'code']]

myresults = check_hypothesis(this_data, abundant_codes, som_cols, stats.spearmanr)

srho_results_2000 = pd.DataFrame.from_dict(myresults, orient='index')
srho_results_2000.fillna("X", inplace=True)

# the survey data with land use attached
srho_results_2000 

,% to buildings,% to trans,% to recreation,% to agg,% to water,% to unproductive,% to woods
G27,0.317386,0.29783,0.316179,-0.24356,0.168906,-0.313578,X
G30,0.186844,0.192731,0.190884,-0.104999,0.215457,-0.129771,X
G67,-0.205994,-0.206335,-0.166623,X,0.116064,X,0.147267
G117,-0.111269,-0.112471,X,-0.124841,X,X,X
G200,0.130511,0.116147,0.158323,-0.128495,0.187634,X,X
G178,0.355357,0.346853,0.320172,-0.261401,0.118711,-0.277548,X
G208,0.118329,0.102087,X,-0.124664,X,-0.12602,X
G941,-0.113281,-0.133638,X,X,X,0.112302,0.173375
G944,-0.111164,-0.113282,X,X,X,0.107774,0.125616
Gfoam,-0.110344,X,-0.107777,-0.102038,0.229041,X,X


### 10000 meters codes with an association to a landuse feature

Where p <= 0.05 for Spearmans test for association.

In [7]:
# the buffer results from qgis
lu_2000 = pd.read_csv('luse_10k.csv')
lu_2000.rename(columns={'location':'slug'}, inplace=True)

# make a df with location slug, landuse label and land use value:
for a_num in lu_2000.AS18_27.unique():
    lu_2000.loc[lu_2000.AS18_27==a_num, "label"]=label_keys[a_num]

# the locations that need land use data
data = pd.DataFrame(index = fd.location.unique())

# records with a quantity greater than 30
abundant_codes = fd[fd.quantity > 30].code.unique()

# getting a list of the current landuse categories
add_these_cols = lu_2000.label.unique()

# counting the number of points inside the buffer for each category and location
bufferdata = lu_2000.groupby(['slug','label'], as_index=False).AS18_27.count()

these_groups = list(group_parts.keys())

some_data = aggregate_buffer_data(data, add_these_cols, bufferdata, label_keys)

sd_0 = adjusted_land_use(some_data, label_keys)

sd_1 = account_for_adj_luse(data, label_keys)

sd_2 = aggregate_the_luse_groups(data, add_these_cols, group_parts, as_1827_part)

fd_luse = assign_luse_stat_to_survey_results(fd, sd_2,som_cols)    

this_data = fd_luse[[unit_label,*som_cols, 'code']]

myresults = check_hypothesis(this_data, abundant_codes, som_cols, stats.spearmanr)

srho_results_2000 = pd.DataFrame.from_dict(myresults, orient='index')
srho_results_2000.fillna("X", inplace=True)

# the survey data with land use attached
srho_results_2000 

,% to buildings,% to trans,% to recreation,% to woods,% to water,% to unproductive,% to agg
G27,0.207443,0.152607,0.14855,-0.103063,0.206567,X,X
G30,0.210692,0.18217,0.16006,X,0.323151,-0.123777,X
G67,-0.183393,-0.218345,-0.200356,0.196207,0.162328,0.148976,X
G112,0.162986,0.173211,0.170703,-0.217958,0.245406,-0.345729,0.245823
G117,-0.132779,-0.147626,-0.166807,0.119796,X,X,X
G178,0.305415,0.273031,0.285807,-0.138495,0.176988,X,-0.161004
G25,0.169099,0.148637,0.153524,X,0.203053,X,X
G944,-0.137606,-0.13415,-0.124112,0.111069,X,0.132843,-0.106867
Gfoam,-0.139906,-0.14746,-0.195902,0.15553,0.219312,0.138448,X
G200,X,X,0.12822,X,0.12018,X,X
